## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Masallatah,LY,32.6167,14.0000,75.13,40,2,11.90,clear sky
1,1,San Luis,AR,-33.2950,-66.3356,44.91,72,69,24.47,broken clouds
2,2,Kapaa,US,22.0752,-159.3190,78.78,73,40,11.50,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,75.00,76,21,6.53,light rain
4,4,Puerto Colombia,CO,10.9878,-74.9547,84.29,76,100,13.38,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
vacation_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Masallatah,LY,32.6167,14.0000,75.13,40,2,11.90,clear sky
2,2,Kapaa,US,22.0752,-159.3190,78.78,73,40,11.50,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,75.00,76,21,6.53,light rain
4,4,Puerto Colombia,CO,10.9878,-74.9547,84.29,76,100,13.38,overcast clouds
5,5,Buala,SB,-8.1450,159.5921,79.32,82,44,4.68,scattered clouds
14,14,Thinadhoo,MV,0.5333,72.9333,83.03,71,66,5.50,broken clouds
22,22,Bambanglipuro,ID,-7.9500,110.2833,76.91,88,100,3.96,overcast clouds
23,23,Atuona,PF,-9.8000,-139.0333,78.26,77,18,18.45,light rain
24,24,Sao Joao Da Barra,BR,-21.6403,-41.0511,77.09,72,0,9.69,clear sky
27,27,Vanimo,PG,-2.6741,141.3028,78.26,83,100,0.56,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_cities_df = vacation_cities_df.dropna()
vacation_cities_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Masallatah,LY,75.13,clear sky,32.6167,14.0000,
2,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,
3,Rikitea,PF,75.00,light rain,-23.1203,-134.9692,
4,Puerto Colombia,CO,84.29,overcast clouds,10.9878,-74.9547,
5,Buala,SB,79.32,scattered clouds,-8.1450,159.5921,
14,Thinadhoo,MV,83.03,broken clouds,0.5333,72.9333,
22,Bambanglipuro,ID,76.91,overcast clouds,-7.9500,110.2833,
23,Atuona,PF,78.26,light rain,-9.8000,-139.0333,
24,Sao Joao Da Barra,BR,77.09,clear sky,-21.6403,-41.0511,
27,Vanimo,PG,78.26,overcast clouds,-2.6741,141.3028,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   213
Country                213
Max Temp               213
Current Description    213
Lat                    213
Lng                    213
Hotel Name             213
dtype: int64

In [17]:
#Drop the rows.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df.count()

City                   213
Country                213
Max Temp               213
Current Description    213
Lat                    213
Lng                    213
Hotel Name             213
dtype: int64

In [18]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Masallatah,LY,75.13,clear sky,32.6167,14.0000,Mustafa Taher Sowaidan
2,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Rikitea,PF,75.00,light rain,-23.1203,-134.9692,People ThankYou
4,Puerto Colombia,CO,84.29,overcast clouds,10.9878,-74.9547,Hotel El Emigrante
5,Buala,SB,79.32,scattered clouds,-8.1450,159.5921,Maringe Lagoon Lodge
22,Bambanglipuro,ID,76.91,overcast clouds,-7.9500,110.2833,Puri Brata
23,Atuona,PF,78.26,light rain,-9.8000,-139.0333,Villa Enata
24,Sao Joao Da Barra,BR,77.09,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
27,Vanimo,PG,78.26,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
28,San Patricio,US,87.69,overcast clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/Vacation_Search.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))